# I lacked the domain knowledge of thermodynamics. So, I tried the possible statistical transformations and feature engineering that I would try when I don't understand features.

1. I have normalised the target column.
2. I have introduced polynomial features with the intention of explaining the target variable better from the features that explained the target variable better. But, I got better performance with polynomial features from all the 4 independent features.
3. I have also tried different transformations of the data. ranging from boxcox transformation to standardising features to the same sclale. but, nothing gave significant performance improvement.
4. I have tried ensemble of multiple algorithms with different hyperparametes, but wierdly the simplest one worked the best. Clearly, presence of a bit of domain knowledge would have hugely helped.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install ppscore
import ppscore as pps

     |████████████████████████████████| 51kB 3.1MB/s 
  Created wheel for ppscore: filename=ppscore-0.0.4-py2.py3-none-any.whl size=10575 sha256=fb1e64a09e17d6c8809f6774957503f19dcaaf7adef8fd46234b4f071a87f078
  Stored in directory: /root/.cache/pip/wheels/0c/2f/cc/0df0a0bb6ace01a72ff490179e083aafa4d9f1b5325641df31
Successfully built ppscore


In [ ]:
train_data = pd.read_csv('/content/drive/My Drive/do not delete/power plant energy output prediction/Train.csv')
test_data = pd.read_csv('/content/drive/My Drive/do not delete/power plant energy output prediction/Test.csv')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


In [ ]:
full_data = pd.concat([train_data, test_data])
full_data.reset_index(drop=True, inplace=True)
full_data

,AT,V,AP,RH,PE
0,29.07,72.51,1009.24,58.38,449.371855
1,32.38,67.17,1006.97,60.04,450.861043
2,19.29,67.71,1007.94,63.71,460.894029
3,16.45,41.48,1016.64,45.21,467.577314
4,21.43,46.97,1013.94,61.25,469.805723
...,...,...,...,...,...
47835,14.60,53.82,1016.28,64.83,NaN
47836,29.67,66.51,1015.60,34.10,NaN
47837,20.13,47.03,1012.59,83.03,NaN
47838,27.14,70.32,1007.08,73.08,NaN


In [ ]:
# renaming columns for better understanding.
full_data.rename(mapper={'AT':'temperature', 'V':'exhaust_vacuum',
                         'AP':'ambient_pressure', 'RH':'relative_humidity',
                         'PE':'power_estimated'},
                 axis='columns', inplace=True)

full_data

,temperature,exhaust_vacuum,ambient_pressure,relative_humidity,power_estimated
0,29.07,72.51,1009.24,58.38,449.371855
1,32.38,67.17,1006.97,60.04,450.861043
2,19.29,67.71,1007.94,63.71,460.894029
3,16.45,41.48,1016.64,45.21,467.577314
4,21.43,46.97,1013.94,61.25,469.805723
...,...,...,...,...,...
47835,14.60,53.82,1016.28,64.83,NaN
47836,29.67,66.51,1015.60,34.10,NaN
47837,20.13,47.03,1012.59,83.03,NaN
47838,27.14,70.32,1007.08,73.08,NaN


In [ ]:
full_data.loc[:9567,'power_estimated'] = np.log1p(full_data.loc[:9567,'power_estimated'])

In [ ]:
for col in ['temperature', 'exhaust_vacuum', 'ambient_pressure', 'relative_humidity']:
    full_data.loc[:,col] = np.log(full_data.loc[:,col])

In [ ]:
full_data

,temperature,exhaust_vacuum,ambient_pressure,relative_humidity,power_estimated
0,3.369707,4.283724,6.916953,4.066973,6.110074
1,3.477541,4.207227,6.914701,4.095011,6.113375
2,2.959587,4.215234,6.915664,4.154342,6.135335
3,2.800325,3.725211,6.924258,3.811318,6.149701
4,3.064792,3.849509,6.921599,4.114964,6.154446
...,...,...,...,...,...
47835,2.681022,3.985645,6.923904,4.171768,NaN
47836,3.390136,4.197352,6.923235,3.529297,NaN
47837,3.002211,3.850786,6.920267,4.419202,NaN
47838,3.301009,4.253056,6.914810,4.291555,NaN


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree=3, interaction_only=False,  
                        include_bias=False)
pf_one = pf.fit_transform(full_data.loc[:, full_data.columns != 'power_estimated'])
pf_one

array([[  3.36970671,   4.28372448,   6.91695285, ..., 194.58123654,
        114.40828403,  67.26884713],
       [  3.47754095,   4.20722672,   6.9147011 , ..., 195.7951352 ,
        115.95341896,  68.66971109],
       [  2.95958683,   4.21523388,   6.91566392, ..., 198.68723119,
        119.35435649,  71.69792607],
       ...,
       [  3.00221124,   3.8507857 ,   6.92026668, ..., 211.63598526,
        135.14828396,  86.30412561],
       [  3.30100865,   4.25305625,   6.91481033, ..., 205.19898122,
        127.35311835,  79.03946042],
       [  2.93492013,   4.11529033,   6.92706759, ..., 205.36572083,
        126.8842252 ,  78.39480971]])

In [ ]:
pf_one.shape

(47840, 34)

In [ ]:
poly_features = pd.DataFrame(data=pf_one, columns=['pf_col_'+str(i) for i in range(34)])
poly_features

,pf_col_0,pf_col_1,pf_col_2,pf_col_3,pf_col_4,pf_col_5,pf_col_6,pf_col_7,pf_col_8,pf_col_9,pf_col_10,pf_col_11,pf_col_12,pf_col_13,pf_col_14,pf_col_15,pf_col_16,pf_col_17,pf_col_18,pf_col_19,pf_col_20,pf_col_21,pf_col_22,pf_col_23,pf_col_24,pf_col_25,pf_col_26,pf_col_27,pf_col_28,pf_col_29,pf_col_30,pf_col_31,pf_col_32,pf_col_33
0,3.369707,4.283724,6.916953,4.066973,11.354923,14.434895,23.308102,13.704507,18.350295,29.630320,17.421793,47.844237,28.131063,16.540272,38.262761,48.641363,78.541469,46.180171,61.835114,99.845489,58.706334,161.221046,94.793432,55.735867,78.607610,126.928128,74.630163,204.951528,120.505723,70.853970,330.936330,194.581237,114.408284,67.268847
1,3.477541,4.207227,6.914701,4.095011,12.093291,14.630803,24.046156,14.240568,17.700757,29.091715,17.228640,47.813091,28.315777,16.769115,42.054915,50.879217,83.621493,49.522160,61.555106,101.167631,59.913300,166.271983,98.469274,58.315285,74.471096,122.395442,72.484793,201.160515,119.130894,70.551469,330.613235,195.795135,115.953419,68.669711
2,2.959587,4.215234,6.915664,4.154342,8.759154,12.475351,20.467508,12.295134,17.768197,29.151141,17.511521,47.826407,28.730030,17.258554,25.923477,36.921883,60.575367,36.388518,52.586521,86.275333,51.826867,141.546406,85.029018,51.078188,74.897105,122.878877,73.815157,201.599493,121.103795,72.748840,330.751361,198.687231,119.354356,71.697926
3,2.800325,3.725211,6.924258,3.811318,7.841823,10.431804,19.390177,10.672932,13.877200,25.794326,14.197966,47.945354,26.390553,14.526147,21.959656,29.212447,54.298807,29.887683,38.860676,72.232508,39.758927,134.262596,73.902137,40.677940,51.695503,96.089317,52.890426,178.606577,98.310387,54.112969,331.986016,182.735004,100.582796,55.363771
4,3.064792,3.849509,6.921599,4.114964,9.392949,11.797944,21.213260,12.611508,14.818720,26.644758,15.840591,47.908533,28.482130,16.932927,28.787433,36.158242,65.014225,38.651645,45.416293,81.660637,48.548113,146.829679,87.291798,51.895897,57.044799,102.569240,60.978498,184.424333,109.642218,65.183458,331.603654,197.141881,117.202934,69.678384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47835,2.681022,3.985645,6.923904,4.171768,7.187876,10.685600,18.563136,11.184601,15.885367,27.596225,16.627189,47.940449,28.884925,17.403652,19.270851,28.648325,49.768168,29.986156,42.589012,73.986074,44.577851,128.529376,77.441106,46.659566,63.313437,109.988761,66.270074,191.073618,115.125061,69.364781,331.935076,199.996453,120.501219,72.604007
47836,3.390136,4.197352,6.923235,3.529297,11.493025,14.229597,23.470711,11.964800,17.617766,29.059256,14.813705,47.931181,24.434155,12.455940,38.962923,48.240275,79.568912,40.562303,59.726632,98.514842,50.220479,162.493242,82.835118,42.227336,73.947973,121.971935,62.178337,201.184053,102.558756,52.281969,331.838821,169.163391,86.235398,43.960717
47837,3.002211,3.850786,6.920267,4.419202,9.013272,11.560872,20.776102,13.267378,14.828550,26.648464,17.017400,47.890091,30.582056,19.529346,27.059747,34.708180,62.374248,39.831471,44.518441,80.004318,51.089829,143.776169,91.813793,58.631223,57.101570,102.617524,65.530360,184.414477,117.764945,75.203327,331.412201,211.635985,135.148284,86.304126
47838,3.301009,4.253056,6.914810,4.291555,10.896658,14.039376,22.825849,14.166459,18.088487,29.409077,18.252224,47.814602,29.675287,18.417442,35.969963,46.344100,75.348324,46.763605,59.710254,97.079619,60.250748,157.836415,97.958379,60.796135,76.931355,125.078460,77.627734,203.358192,126.210665,78.330417,330.628904,205.198981,127.353118,79.039460


In [ ]:
full_data_new = pd.concat([full_data, poly_features], join='outer', axis=1)
full_data_new.describe()

,temperature,exhaust_vacuum,ambient_pressure,relative_humidity,power_estimated,pf_col_0,pf_col_1,pf_col_2,pf_col_3,pf_col_4,pf_col_5,pf_col_6,pf_col_7,pf_col_8,pf_col_9,pf_col_10,pf_col_11,pf_col_12,pf_col_13,pf_col_14,pf_col_15,pf_col_16,pf_col_17,pf_col_18,pf_col_19,pf_col_20,pf_col_21,pf_col_22,pf_col_23,pf_col_24,pf_col_25,pf_col_26,pf_col_27,pf_col_28,pf_col_29,pf_col_30,pf_col_31,pf_col_32,pf_col_33
count,47840.000000,47840.000000,47840.000000,47840.000000,9568.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000
mean,2.888112,3.966958,6.920910,4.272367,6.146684,2.888112,3.966958,6.920910,4.272367,8.548158,11.543669,19.987091,12.291092,15.792418,27.454395,16.932355,47.899030,29.568802,18.300911,25.798061,34.392335,59.153916,36.249671,46.295677,79.886068,49.083578,138.320138,85.060475,52.451696,63.089649,109.293414,67.344656,190.005634,117.185430,72.464931,331.505357,204.644102,126.660075,78.586678
std,0.454940,0.235935,0.005857,0.218618,0.036275,0.454940,0.235935,0.005857,0.218618,2.453105,2.357355,3.141079,1.757939,1.873036,1.623460,1.117607,0.081073,1.515708,1.823869,10.253525,11.340889,16.955047,9.738577,11.784599,16.284305,9.290965,21.687544,12.133381,7.701326,11.177982,12.924661,7.723122,11.172442,7.705697,7.269142,0.841743,10.511463,12.633031,11.457018
min,0.593327,3.233173,6.900620,3.241029,6.074210,0.593327,3.233173,6.900620,3.241029,0.352037,2.180045,4.114318,2.577065,10.453408,22.360597,12.771408,47.618555,22.440444,10.504267,0.208873,1.293479,2.441135,1.529042,8.010081,15.117128,9.468842,28.529993,17.870193,11.193265,33.797679,72.295682,44.262445,154.645695,88.415687,42.921160,328.597546,155.374602,72.730122,34.044629
25%,2.603430,3.731460,6.916814,4.148320,6.115658,2.603430,3.731460,6.916814,4.148320,6.777849,9.712085,18.024998,11.265188,13.923793,25.823903,16.191559,47.842318,28.704828,17.208556,17.645655,25.330825,46.931722,29.417968,36.156899,67.249729,42.107092,124.796718,78.005226,47.895114,51.956075,96.347122,61.018917,178.824090,112.091261,68.019257,330.916418,198.654205,119.068194,71.386592
50%,3.012835,3.952781,6.920612,4.317155,6.140702,3.012835,3.952781,6.920612,4.317155,9.077176,12.046465,20.850983,12.782700,15.624478,27.353588,16.840012,47.894874,29.882186,18.637825,27.348034,36.363077,62.817204,38.979651,47.958706,83.376669,51.448262,144.332650,88.472811,53.223211,61.760138,108.093491,66.626655,189.253078,116.583787,72.973135,331.461854,206.803150,129.016607,80.462374
75%,3.247269,4.197803,6.924868,4.440649,6.177043,3.247269,4.197803,6.924868,4.440649,10.544755,13.546127,22.465608,13.635379,17.621552,29.039240,17.730952,47.953797,30.738641,19.719366,34.241656,43.874243,72.953796,44.146248,56.820124,93.709773,56.613962,155.440326,94.328512,58.262759,73.971810,121.896822,73.554612,200.926899,122.707506,77.307430,332.073716,212.776451,136.474899,87.566787
max,3.613886,4.401339,6.940513,4.606769,6.235159,3.613886,4.401339,6.940513,4.606769,13.060175,15.454079,24.986288,15.149706,19.371785,30.432195,19.949056,48.170719,31.916018,21.222320,47.197991,55.286812,90.297608,52.400080,67.754918,106.911358,66.015893,172.754345,104.756894,69.406632,85.261790,133.942403,87.146150,210.417438,138.034263,91.100104,334.329491,221.193171,146.978694,97.766323


# modelling

In [ ]:
train_new = full_data_new.loc[:9_567,:]
test_new = full_data_new.loc[9_568:,full_data_new.columns != 'power_estimated']
train_data.shape, train_new.shape, test_data.shape, test_new.shape

((9568, 5), (9568, 39), (38272, 4), (38272, 38))

In [ ]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train_new.loc[:, train_new.columns != 'power_estimated'],
                                            train_new.loc[:,'power_estimated'],
                                            test_size=0.05, random_state=100, shuffle=True)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((9089, 38), (479, 38), (9089,), (479,))

## XG Boost

In [ ]:
xgb_one = XGBRegressor(max_depth=6, learning_rate=0.01, n_estimators=10_000, colsample_bytree=0.85, seed=100)
xgb_one.fit(X=X_tr, y=y_tr, eval_set=[(X_val, y_val)], eval_metric='rmse', early_stopping_rounds=500, verbose=100)

[08:10:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:5.59048
Will train until validation_0-rmse hasn't improved in 500 rounds.
[100]	validation_0-rmse:2.04684
[200]	validation_0-rmse:0.749456
[300]	validation_0-rmse:0.274682
[400]	validation_0-rmse:0.101013
[500]	validation_0-rmse:0.03795
[600]	validation_0-rmse:0.015817
[700]	validation_0-rmse:0.009084
[800]	validation_0-rmse:0.007517
[900]	validation_0-rmse:0.007086
[1000]	validation_0-rmse:0.006877
[1100]	validation_0-rmse:0.006688
[1200]	validation_0-rmse:0.006544
[1300]	validation_0-rmse:0.006414
[1400]	validation_0-rmse:0.006281
[1500]	validation_0-rmse:0.00619
[1600]	validation_0-rmse:0.006126
[1700]	validation_0-rmse:0.00608
[1800]	validation_0-rmse:0.006025
[1900]	validation_0-rmse:0.005975
[2000]	validation_0-rmse:0.005922
[2100]	validation_0-rmse:0.005868
[2200]	validation_0-rmse:0.005817
[2300]	validation_0-rmse:0.005768
[2400]	v

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.85, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=10000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=100,
             silent=None, subsample=1, verbosity=1)

In [ ]:
xgb_predict_one = xgb_one.predict(data=test_new, validate_features=True)
xgb_predict_one = np.expm1(xgb_predict_one)
xgb_predict_one = pd.Series(xgb_predict_one)
xgb_predict_one = xgb_predict_one.rename('PE')
xgb_predict_one

0        472.062653
1        494.926666
2        495.702423
3        480.256409
4        446.141357
            ...    
38267    476.026031
38268    446.516968
38269    471.167755
38270    445.263550
38271    466.774323
Name: PE, Length: 38272, dtype: float32

In [ ]:
xgb_predict_one.to_csv(path_or_buf='/content/drive/My Drive/do not delete/power plant energy output prediction/three_xgb_predict_with_pf_unrounded.csv', index=False, float_format='%.7f')